# 프로젝트 : 멋진 작사가 만들기 

## STEP1. 데이터 다운로드
`~/aiffel/lyricist/data/lyrics`에 데이터 넣어 놓음

## STEP2. 데이터 읽어오기 
`glob` 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이

`glob` 를 활용하여 모든 `txt` 파일을 읽어온 후, `raw_corpus` 리스트에 문장 단위로 저장

In [ ]:
import glob, re, os

txt_file_path = '/content/drive/MyDrive/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


## Step 3. 데이터 정제
앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하기

`preprocess_sentence()` 함수를 활용해 데이터를 정제

추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거

문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기

In [ ]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence


print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


문장 길이 0인것 제외, 문장 끝이 `:` 로 끝나는 문장을 제외하진 않음 (가사여서)

In [ ]:
corpus = []
for sentence in raw_corpus:
  if len(sentence) == 0: continue
  if len(sentence.split(' ')) > 15: continue
  corpus.append(preprocess_sentence(sentence))

print(len(corpus))
corpus[:10]

168535


['<start> looking for some education <end>',
 '<start> made my way into the night <end>',
 '<start> all that bullshit conversation <end>',
 '<start> baby , can t you read the signs ? i won t bore you with the details , baby <end>',
 '<start> i don t even wanna waste your time <end>',
 '<start> let s just say that maybe <end>',
 '<start> you could help me ease my mind <end>',
 '<start> i ain t mr . right but if you re looking for fast love <end>',
 '<start> if that s love in your eyes <end>',
 '<start> it s more than enough <end>']

> 데이터의 수가 많다. 

In [ ]:
corpus = []
for sentence in raw_corpus:
  if len(sentence) == 0: continue
  corpus.append(preprocess_sentence(sentence))
print(len(corpus))
corpus[:10]

len_corpus = []
for sentence in corpus:
  if len(sentence.split()) <= 15:
    len_corpus.append(sentence)
  
print(len(len_corpus))

175986
156227


여기서 토큰이 15개 이상인 문장을 날리고 싶어 `if len(sentence.split(' ')) > 15: continue` 을 사용했으나 생각보다 데이터의 수가 많아서 1차로 `corpus`에 길이가 0인것만 제외하고 `preprocess_sentence`를 해주고 2차로 `len_corpus`로 토큰의 길이가 15이하인 것만 가져왔다. 

In [ ]:
li1 = set(corpus)
li2 = set(len_corpus)

In [ ]:
x = next(iter(li1-li2))
print(len(x.split()))
print(x)

16
<start> how many times can i say the same thing different ways that rhyme ? <end>


##Step 4. 평가 데이터셋 분리
훈련 데이터와 평가 데이터를 분리

`tokenize()` 함수로 데이터를 Tensor로 변환한 후, `sklearn` 모듈의 `train_test_split()` 함수를 사용해 훈련 데이터와 평가 데이터를 분리 

단어장의 크기는 12,000 이상 으로 설정하고 총 데이터의 20% 를 평가 데이터셋으로 사용

### Tensor 변환 

In [ ]:
import tensorflow as tf

def tokenize(len_corpus):
  tokenizer = tf.keras.preprocessing.text.Tokenizer(
      num_words = 12000,
      filters = ' ',
      oov_token = "<unk>"
  )
  tokenizer.fit_on_texts(len_corpus)
  tensor = tokenizer.texts_to_sequences(len_corpus)
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding = 'post')

  print(tensor, tokenizer)
  return tensor, tokenizer

tensor, tokenizer = tokenize(len_corpus)

[[  2 291  28 ...   0   0   0]
 [  2 219  13 ...   0   0   0]
 [  2  25  15 ...   0   0   0]
 ...
 [  2  21  77 ...   0   0   0]
 [  2  42  26 ...   0   0   0]
 [  2  21  77 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fbb1d22cc10>


In [ ]:
print(tensor[:3, :10])

[[   2  291   28   94 4490    3    0    0    0    0]
 [   2  219   13   86  220    6  113    3    0    0]
 [   2   25   15 1039 2250    3    0    0    0    0]]


In [ ]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [ ]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2  291   28   94 4490    3    0    0    0    0    0    0    0    0]
[ 291   28   94 4490    3    0    0    0    0    0    0    0    0    0]


### 데이터셋 분리 

In [ ]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2, random_state = 42)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124981, 14)
Target Train: (124981, 14)


In [ ]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [ ]:
BUFFER_SIZE = len(enc_val)
BATCH_SIZE = 256
steps_per_epoch = len(enc_val) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
valset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
valset = valset.shuffle(BUFFER_SIZE)
valset = valset.batch(BATCH_SIZE, drop_remainder=True)
valset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

여기까지 올바르게 진행했을 경우, 아래 실행 결과를 확인할 수 있음

In [ ]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
# out:

# Source Train: (124960, 14)
# Target Train: (124960, 14)

Source Train: (124981, 14)
Target Train: (124981, 14)


## Step 5. 인공지능 만들기
모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하기 (Loss는 아래 제시된 Loss 함수를 그대로 사용!)

모델이 생성한 가사 한 줄을 제출하기

In [ ]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

* batch size = 모델이 경사하강법을 통해 손실/오차를 계산해서 모델의 가중치를 업데이트할 때 한번에 몇 개의 관측치를 사용하는지를 결정하는 파라미터 (대체적으로 32~512 사이의 2의 제곱수 사용)

  가중치를 업데이트 할 수 있을 만큼의 충분한 정보를 제공할 수 있는 충분한 양의 관측치의 크기를 확인하기 위한 변수

  * 너무 큰 배치를 고를 경우 
    1. 모든 데이터에 대한 Loss를 계산해야 하는 문제점
    2. 주어진 epoch 안에 가중치를 충분히 업데이트 할 만큼의 iteration을 돌릴 수 없음(학습 효과 저하)

  * 너무 작은 배치 크기를 고르는 경우
    1. 학습에 오랜 시간
    2. 추정값에 노이즈 증가 

In [ ]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 2.01889692e-04,  4.53735774e-05,  1.70246276e-04, ...,
         -6.91563546e-05,  6.59728612e-05,  6.05093519e-05],
        [ 3.69252230e-04,  1.34047063e-04,  2.42031660e-04, ...,
         -1.21333069e-04,  4.97304136e-04, -8.28525299e-05],
        [ 4.76705842e-04,  2.84860056e-04,  5.82214197e-05, ...,
          7.65052609e-05,  7.96879933e-04,  1.02770891e-05],
        ...,
        [ 1.86874051e-04, -1.90771912e-04,  6.61871862e-04, ...,
         -1.95370824e-03,  2.80330569e-04,  1.16308499e-03],
        [ 4.81237803e-04, -2.58714572e-04,  5.46451251e-04, ...,
         -2.28936831e-03, -8.44739334e-05,  1.32249552e-03],
        [ 8.32312973e-04, -2.80675857e-04,  4.26743994e-04, ...,
         -2.57522380e-03, -4.11504559e-04,  1.48326496e-03]],

       [[ 2.01889692e-04,  4.53735774e-05,  1.70246276e-04, ...,
         -6.91563546e-05,  6.59728612e-05,  6.05093519e-05],
        [ 1.60443029e-04,  2.26515418e-04,  4

In [ ]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, batch_size=256, validation_data=(valset), epochs=10)

Epoch 1/10
488/488 [==============================] - 187s 375ms/step - loss: 3.5099 - val_loss: 3.1696
Epoch 2/10
488/488 [==============================] - 183s 376ms/step - loss: 3.0269 - val_loss: 2.9665
Epoch 3/10
488/488 [==============================] - 184s 377ms/step - loss: 2.8583 - val_loss: 2.8538
Epoch 4/10
488/488 [==============================] - 183s 374ms/step - loss: 2.7336 - val_loss: 2.7725
Epoch 5/10
488/488 [==============================] - 183s 374ms/step - loss: 2.6291 - val_loss: 2.7087
Epoch 6/10
488/488 [==============================] - 183s 375ms/step - loss: 2.5344 - val_loss: 2.6600
Epoch 7/10
488/488 [==============================] - 180s 369ms/step - loss: 2.4468 - val_loss: 2.6119
Epoch 8/10
488/488 [==============================] - 182s 373ms/step - loss: 2.3644 - val_loss: 2.5730
Epoch 9/10
488/488 [==============================] - 182s 373ms/step - loss: 2.2864 - val_loss: 2.5424
Epoch 10/10
488/488 [==============================] - 182s 373m

In [ ]:
embedding_size = 512
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, batch_size=256, validation_data=(valset), epochs=10)

Epoch 1/10
488/488 [==============================] - 519s 1s/step - loss: 3.2765 - val_loss: 2.9188
Epoch 2/10
488/488 [==============================] - 503s 1s/step - loss: 2.7419 - val_loss: 2.6691
Epoch 3/10
488/488 [==============================] - 504s 1s/step - loss: 2.4396 - val_loss: 2.4863
Epoch 4/10
488/488 [==============================] - 502s 1s/step - loss: 2.1418 - val_loss: 2.3497
Epoch 5/10
488/488 [==============================] - 501s 1s/step - loss: 1.8573 - val_loss: 2.2502
Epoch 6/10
488/488 [==============================] - 501s 1s/step - loss: 1.5988 - val_loss: 2.1815
Epoch 7/10
488/488 [==============================] - 502s 1s/step - loss: 1.3826 - val_loss: 2.1426
Epoch 8/10
488/488 [==============================] - 501s 1s/step - loss: 1.2142 - val_loss: 2.1343
Epoch 9/10
488/488 [==============================] - 502s 1s/step - loss: 1.0991 - val_loss: 2.1462
Epoch 10/10
488/488 [==============================] - 502s 1s/step - loss: 1.0298 - val_lo

In [ ]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> i hate", max_len=20)

'<start> i hate you <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m not gonna crack <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> i like", max_len=20)

'<start> i like the way how you re touchin me <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> it is", max_len=20)

'<start> it is a weeping and a moaning and a gnashing of teeth <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> i", max_len=20)

'<start> i m not gonna lose any sleep tryna know where you are <end> '

# 회고 

이번 프로젝트를 진행하면서 첫번째로 당황(?)스러웠던 것은 토큰 개수 15개 이하의 문장만 가져오는 것이었다 .

```python
corpus = []
for sentence in raw_corpus:
  if len(sentence) == 0: continue
  if len(sentence.split(' ')) > 15: continue
  corpus.append(preprocess_sentence(sentence))
```
```python
corpus = []
for sentence in raw_corpus:
  if len(sentence) == 0: continue
  corpus.append(preprocess_sentence(sentence))
print(len(corpus))
corpus[:10]

len_corpus = []
for sentence in corpus:
  if len(sentence.split()) <= 15:
    len_corpus.append(sentence)
```

처음엔 첫 번째 코드로 진행했는데 train.shape이 LMS 기준과 달라 두 번째 코드로 진행하니 기준과 매우 비슷해졌다.

처음엔 두 코드가 같은 코드라고 생각했다. 팀원들에게 물어보고 토론한 뒤 첫번째 `corpus`코드는 길이가 15 미만인 문장을 가져와 `preprocess_sentence`를 해주기 때문에 `<start>`와 `<end>`가 붙어 길이가 늘어나는 것을 알게 되었다. 

두 번째로 당황스러웠던 점은 model을 돌리는데 시간이 너무 많이 소요된다는 점이었다. 

루브릭 기준을 맞추기 위해 val_loss를 2.2 이하로 낮췄어야 하는데, 첫 번째 시도는 `embedding_size = 256, hidden_size = 1024`으로 하이퍼파라미터를 설정하고, `loss: 2.2125 - val_loss: 2.5138` 정도의 loss를 보였다. 

두 번째 시도에서는 `embedding_size = 512, hidden_size = 2048`로 하이퍼파라미터 값을 두배로 설정하였더니 `loss: 1.0298 - val_loss: 2.1615` 정도의 loss를 보였다. 분명 loss는 하락했지만, 소요되는 시간이 2배정도 걸렸고 `loss`와 `validation loss`의 차이가 많이 났다. 

두 번째 모형은 overfitting된 모형으로 해석을 했는데 앞으로 더 배워가면서 하이퍼파라미터를 높여 시간이 많이 소요되는 모형 대신, 전처리 과정을 통해 loss를 줄이는 모형을 만들고 싶다. 

[딥러닝을 이용한 자연어 처리 입문](https://wikidocs.net/21698)

[Recurrent Networks](https://younnggsuk.github.io/2020/12/23/eecs-12.html)

[BackPropagation through LSTM: A differential approach](https://medium.com/@raman.shinde15/backpropagation-through-lstm-a-differential-approach-4eb5ecc58d9d)

[딥러닝 RNN - LSTM을 활용한 텍스트 분류 모델](https://blog.naver.com/mini_s0n/222326186376)

[RNN과 LSTM을 이해해보자!](https://ratsgo.github.io/natural%20language%20processing/2017/03/09/rnnlstm/)

[정규식 : 괄호안에 문자, 문장 제거하기](https://snepbnt.tistory.com/378)